# Protein MD Setup tutorial using BioExcel Building Blocks (biobb)
**Based on the official GROMACS tutorial:** [http://www.mdtutorials.com/gmx/lysozyme/index.html](http://www.mdtutorials.com/gmx/lysozyme/index.html)
***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Lysozyme** protein (PDB code 1AKI). 
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_md](https://github.com/bioexcel/biobb_md): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup.git
cd biobb_wf_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_MDsetup_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_md_setup/notebooks/biobb_MDsetup_tutorial.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create Solvent Box](#box)
 6. [Fill the Box with Water Molecules](#water)
 7. [Adding Ions](#ions)
 8. [Energetically Minimize the System](#min)
 9. [Equilibrate the System (NVT)](#nvt)
 10. [Equilibrate the System (NPT)](#npt)
 11. [Free Molecular Dynamics Simulation](#free)
 12. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 13. [Output Files](#output)
 14. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)

In [1]:
import nglview
import ipywidgets

<a id="fetch"></a>
***
## Fetching PDB structure
the mutated Asparaginase 3D structure is stored in the `3ECA_mut.pdb` file

we can visualise it before making further changes to it.
***

In [2]:
from pathlib import Path
working_dir = "temp/6V23"
Path(working_dir).mkdir(parents=True, exist_ok=True)

In [3]:
import os
os.chdir(working_dir)
os.getcwd()

'/home/ma/git/druglord/temp/6V23'

In [4]:
pdbCode = "6V23"

In [5]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode,
}

# Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2022-04-26 19:27:58,798 [MainThread  ] [INFO ]  Downloading: 6v23 from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb6v23.ent


/home/ma/miniconda3/envs/druglord/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2022-04-26 19:28:01,833 [MainThread  ] [INFO ]  Writting pdb to: 6V23.pdb
2022-04-26 19:28:01,835 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**:    

In [6]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **PDB resolution** idealy beloew 2. The simulation result will be severely affected if it's over 4.
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, existence of **water molecules**, **heteroatoms**, **modified residues** and possible **atomic clashes**.


***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [7]:
# Check & Fix PDB
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = pdbCode + '_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=downloaded_pdb, 
             output_pdb_path=fixed_pdb)

2022-04-26 19:28:03,028 [MainThread  ] [INFO ]  check_structure -i 6V23.pdb -o 6V23_fixed.pdb --force_save fixside --fix ALL

2022-04-26 19:28:03,032 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:28:03,033 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure 6V23.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  315
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  2357

Running fixside. Options: --fix ALL
3 Residues with missing side chain atoms found
 HIS A-2    CG,ND1,CE1,NE2,CD2
 HIS A-1    CG,ND1,CE1,NE2,CD2
 LYS A107   NZ
Fixing side chains
HIS A-2
  Adding new atom CG
  Adding new atom ND1

0

### Visualizing 3D structure
Visualizing the fixed **PDB structure** using **NGL**. In this particular example, the checking step didn't find any issue to be solved, so there is no difference between the original structure and the fixed one.   

In [8]:
# Show protein
view = nglview.show_structure_file(fixed_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_md.gromacs.pdb2gmx**
***

In [9]:
# Create system topology
# Import module
from biobb_md.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = pdbCode+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode+'_pdb2gmx_top.zip'

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb, 
        output_gro_path=output_pdb2gmx_gro, 
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties={'ignh':'yes'})

2022-04-26 19:28:04,146 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2022-04-26 19:28:04,148 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:28:05,594 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f 6V23_fixed.pdb -o 6V23_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp -ignh

2022-04-26 19:28:05,596 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:28:05,597 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading 6V23_fixed.pdb...
Read '', 2379 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 315 residues with 2379 atoms

  chain  #res #atoms
  1 'A'   315   2379  

Reading residue database... (Amber99sb-ildn)
Processing chain 1 'A' (2379 atoms, 315 residue

0

### Visualizing 3D structure
Visualizing the generated **GRO structure** using **NGL**. Note that **hydrogen atoms** were added to the structure by the **pdb2gmx GROMACS tool** when generating the **topology**.    

In [10]:
# Show protein
view = nglview.show_structure_file(output_pdb2gmx_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **cubic box** is used to define the unit cell, with a **distance from the protein to the box edge of 1.0 nm**. The protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_md.gromacs.editconf** 
***

In [11]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = pdbCode+'_editconf.gro'

prop = {
    'box_type': 'cubic',
    'distance_to_molecule': 1.0
}

#Create and launch bb
editconf(input_gro_path=output_pdb2gmx_gro, 
         output_gro_path=output_editconf_gro,
         properties=prop)

2022-04-26 19:28:06,558 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2022-04-26 19:28:06,560 [MainThread  ] [INFO ]  Centering molecule in the box.
2022-04-26 19:28:06,561 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2022-04-26 19:28:06,562 [MainThread  ] [INFO ]  Box type: cubic
2022-04-26 19:28:06,562 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:28:06,617 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f 6V23_pdb2gmx.gro -o 6V23_editconf.gro -d 1.0 -bt cubic -c

2022-04-26 19:28:06,620 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:28:06,621 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 4721 atoms
Volume: 176.366 nm^3, corresponds to roughly 79300 electrons
No velocities found
    system size :  5.024  6.365  5.516 (nm)
    diameter    :  7.735               (nm)
    center      :  1.352  0.804  4.973 (nm)
    box vectors :  5.024  6.364 

0

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein structure system** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_md.gromacs.solvate** 
***

In [12]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro = pdbCode+'_solvate.gro'
output_solvate_top_zip = pdbCode+'_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro, 
        output_gro_path=output_solvate_gro, 
        input_top_zip_path=output_pdb2gmx_top_zip, 
        output_top_zip_path=output_solvate_top_zip)

2022-04-26 19:28:06,890 [MainThread  ] [INFO ]  GROMACS Solvate 20191 version detected
2022-04-26 19:28:06,899 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_pdb2gmx_top.zip
2022-04-26 19:28:06,900 [MainThread  ] [INFO ]  to:
2022-04-26 19:28:06,901 [MainThread  ] [INFO ]  ['c695e642-ffe7-4d1f-80d4-dd1d74d33581/p2g.top', 'c695e642-ffe7-4d1f-80d4-dd1d74d33581/posre.itp']
2022-04-26 19:28:06,902 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:28:06,903 [MainThread  ] [INFO ]  6V23_pdb2gmx_top.zip
2022-04-26 19:28:06,903 [MainThread  ] [INFO ]  To: 
2022-04-26 19:28:06,904 [MainThread  ] [INFO ]  c695e642-ffe7-4d1f-80d4-dd1d74d33581/p2g.top
2022-04-26 19:28:06,905 [MainThread  ] [INFO ]  c695e642-ffe7-4d1f-80d4-dd1d74d33581/posre.itp
2022-04-26 19:28:06,905 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:28:08,046 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright solvate -cp 6V23_editconf.gro -cs spc216.gro -o 6V23_solvate.gro -p c695e642-ffe7-4d

0

### Visualizing 3D structure
Visualizing the **protein system** with the newly added **solvent box** using **NGL**.<br> Note the **cubic box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the cube.

In [13]:
# Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='green')
view.add_representation(repr_type='ball+stick', selection='SOL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein structure** charge
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_md.gromacs.genion** 
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation
A simple **energy minimization** molecular dynamics parameters (mdp) properties will be used to generate the portable binary run file for **ion generation**, although **any legitimate combination of parameters** could be used in this step.

In [14]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppion_tpr = pdbCode+'_gppion.tpr'
prop = {
    'simulation_type': 'minimization',
    'maxwarn': 1
}

# Create and launch bb
grompp(input_gro_path=output_solvate_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,  
       properties=prop)

2022-04-26 19:28:20,291 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-26 19:28:20,301 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_solvate_top.zip
2022-04-26 19:28:20,302 [MainThread  ] [INFO ]  to:
2022-04-26 19:28:20,303 [MainThread  ] [INFO ]  ['a43b9467-4d16-4130-b443-a98b508ebfc2/p2g.top', 'a43b9467-4d16-4130-b443-a98b508ebfc2/posre.itp']
2022-04-26 19:28:20,304 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:28:20,304 [MainThread  ] [INFO ]  6V23_solvate_top.zip
2022-04-26 19:28:20,305 [MainThread  ] [INFO ]  To: 
2022-04-26 19:28:20,306 [MainThread  ] [INFO ]  a43b9467-4d16-4130-b443-a98b508ebfc2/p2g.top
2022-04-26 19:28:20,307 [MainThread  ] [INFO ]  a43b9467-4d16-4130-b443-a98b508ebfc2/posre.itp
2022-04-26 19:28:20,308 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:28:21,003 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f e5d29982-f249-44ab-acb1-e39125cc30b7/grompp.mdp -c 6V23_solvate.gro -r 6V23

0

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [15]:
# Genion: Adding ions to neutralize the system
from biobb_md.gromacs.genion import genion

# Create prop dict and inputs/outputs
output_genion_gro = pdbCode+'_genion.gro'
output_genion_top_zip = pdbCode+'_genion_top.zip'
prop={
    'neutral':True
}

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr, 
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip, 
       output_top_zip_path=output_genion_top_zip, 
       properties=prop)

2022-04-26 19:28:21,301 [MainThread  ] [INFO ]  GROMACS Genion 20191 version detected
2022-04-26 19:28:21,309 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_solvate_top.zip
2022-04-26 19:28:21,310 [MainThread  ] [INFO ]  to:
2022-04-26 19:28:21,311 [MainThread  ] [INFO ]  ['b05592bb-36ae-4791-8eef-a80e436473db/p2g.top', 'b05592bb-36ae-4791-8eef-a80e436473db/posre.itp']
2022-04-26 19:28:21,312 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:28:21,312 [MainThread  ] [INFO ]  6V23_solvate_top.zip
2022-04-26 19:28:21,313 [MainThread  ] [INFO ]  To: 
2022-04-26 19:28:21,314 [MainThread  ] [INFO ]  b05592bb-36ae-4791-8eef-a80e436473db/p2g.top
2022-04-26 19:28:21,314 [MainThread  ] [INFO ]  b05592bb-36ae-4791-8eef-a80e436473db/posre.itp
2022-04-26 19:28:21,315 [MainThread  ] [INFO ]  To reach up 0.05 mol/litre concentration
2022-04-26 19:28:21,316 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:28:21,575 [MainThread  ] [INFO ]  echo "SOL" | gmx -nobac

0

### Visualizing 3D structure
Visualizing the **neutralized protein system** with the newly added **ions** using **NGL**

In [16]:
# Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
The **minimization** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **energy minimization**:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.01 ; Minimization step size (nm)
-  nsteps      = 50000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, but the **maximum force** is placed at **500 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5,000 steps**. 

In [17]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppmin_tpr = pdbCode+'_gppmin.tpr'
prop = {
    'mdp':{
        'emtol':'500',
        'nsteps':'5000'
    },
    'simulation_type': 'minimization'
}

# Create and launch bb
grompp(input_gro_path=output_genion_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppmin_tpr,  
       properties=prop)

2022-04-26 19:28:36,419 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-26 19:28:36,429 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_genion_top.zip
2022-04-26 19:28:36,430 [MainThread  ] [INFO ]  to:
2022-04-26 19:28:36,431 [MainThread  ] [INFO ]  ['407b3184-8881-4c95-a4e2-041055857a1a/p2g.top', '407b3184-8881-4c95-a4e2-041055857a1a/posre.itp']
2022-04-26 19:28:36,431 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:28:36,432 [MainThread  ] [INFO ]  6V23_genion_top.zip
2022-04-26 19:28:36,433 [MainThread  ] [INFO ]  To: 
2022-04-26 19:28:36,434 [MainThread  ] [INFO ]  407b3184-8881-4c95-a4e2-041055857a1a/p2g.top
2022-04-26 19:28:36,435 [MainThread  ] [INFO ]  407b3184-8881-4c95-a4e2-041055857a1a/posre.itp
2022-04-26 19:28:36,436 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:28:37,160 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f 56745072-1816-4983-a25d-04dffd5f6f0d/grompp.mdp -c 6V23_genion.gro -r 6V23_ge

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step. 

In [18]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = pdbCode+'_min.trr'
output_min_gro = pdbCode+'_min.gro'
output_min_edr = pdbCode+'_min.edr'
output_min_log = pdbCode+'_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr, 
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro, 
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log)

2022-04-26 19:28:37,417 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-26 19:28:37,419 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:29:40,071 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 6V23_gppmin.tpr -o 6V23_min.trr -c 6V23_min.gro -e 6V23_min.edr -g 6V23_min.log

2022-04-26 19:29:40,073 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:29:40,074 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /home/ma/miniconda3/envs/druglord/bin/gmx
Data prefix:  /home/ma/miniconda3/envs/druglord
Working dir:  /home/ma/git/druglord/temp/6V23
Command line:
  gmx -nobackup -nocopyright mdrun -s 6V23_gppmin.tpr -o 6V23_min.trr -c 6V23_min.gro -e 6V23_min.edr -g 6V23_min.log

Compiled SIMD: SSE2, but for this host/run AVX_512 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This might affect your simulation
speed as accu

0

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [19]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbCode+'_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop)

2022-04-26 19:29:40,452 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:29:40,483 [MainThread  ] [INFO ]  gmx energy -f /home/ma/git/druglord/temp/6V23/6V23_min.edr -o 6V23_min_ene.xvg -xvg none < afc85d98-e677-4d92-8fb9-8826867d752c/instructions.in

2022-04-26 19:29:40,484 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:29:40,485 [MainThread  ] [INFO ]  
Statistics over 3136 steps [ 0.0000 through 3135.0000 ps ], 1 data sets
All statistics are over 2482 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                -1.56179e+06      24000    73610.6    -158039  (kJ/mol)

2022-04-26 19:29:40,485 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau 

0

In [20]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein system** in **NVT ensemble** (constant Number of particles, Volume and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNVTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNVTStep2): Equilibrate the **protein system** with **NVT** ensemble.
- [Step 3](#eqNVTStep3): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 
***
**Building Blocks** used:
- [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
- [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
- [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNVTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NVT)
The **nvt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NVT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 50,000
-  pcoupl                   = no
-  gen_vel                  = yes
-  gen_temp                 = 300
-  gen_seed                 = -1

In this particular example, the default parameters will be used: **md** integrator algorithm, a **step size** of **2fs**, **50,000 equilibration steps** with the protein **heavy atoms restrained** (100ps in total), and a temperature of **300K**.

*Please refer to the [original example](http://www.mdtutorials.com/gmx/lysozyme/06_equil.html) for further details*

In [21]:
# Grompp: Creating portable binary run file for NVT Equilibration
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnvt_tpr = pdbCode+'_gppnvt.tpr'
prop = {
    'mdp':{
        'nsteps': 50000,
        'dt': 0.002,
        'Define': '-DPOSRES',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    },
    'simulation_type': 'nvt'
}

# Create and launch bb
grompp(input_gro_path=output_min_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppnvt_tpr,  
       properties=prop)

2022-04-26 19:29:41,447 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-26 19:29:41,455 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_genion_top.zip
2022-04-26 19:29:41,455 [MainThread  ] [INFO ]  to:
2022-04-26 19:29:41,456 [MainThread  ] [INFO ]  ['7ab5b89a-b1a1-4a16-a190-b8fc2f19f19a/p2g.top', '7ab5b89a-b1a1-4a16-a190-b8fc2f19f19a/posre.itp']
2022-04-26 19:29:41,457 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:29:41,458 [MainThread  ] [INFO ]  6V23_genion_top.zip
2022-04-26 19:29:41,458 [MainThread  ] [INFO ]  To: 
2022-04-26 19:29:41,459 [MainThread  ] [INFO ]  7ab5b89a-b1a1-4a16-a190-b8fc2f19f19a/p2g.top
2022-04-26 19:29:41,459 [MainThread  ] [INFO ]  7ab5b89a-b1a1-4a16-a190-b8fc2f19f19a/posre.itp
2022-04-26 19:29:41,460 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:29:42,218 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f ba8fe870-854a-47fa-bb4c-16c698b52f8a/grompp.mdp -c 6V23_min.gro -r 6V23_min.g

0

<a id="eqNVTStep2"></a>
### Step 2: Running NVT equilibration

In [22]:
# Mdrun: Running Equilibration NVT
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_nvt_trr = pdbCode+'_nvt.trr'
output_nvt_gro = pdbCode+'_nvt.gro'
output_nvt_edr = pdbCode+'_nvt.edr'
output_nvt_log = pdbCode+'_nvt.log'
output_nvt_cpt = pdbCode+'_nvt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnvt_tpr, 
      output_trr_path=output_nvt_trr, 
      output_gro_path=output_nvt_gro, 
      output_edr_path=output_nvt_edr, 
      output_log_path=output_nvt_log, 
      output_cpt_path=output_nvt_cpt)

2022-04-26 19:29:42,621 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-26 19:29:42,623 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:34:00,597 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 6V23_gppnvt.tpr -o 6V23_nvt.trr -c 6V23_nvt.gro -e 6V23_nvt.edr -g 6V23_nvt.log -cpo 6V23_nvt.cpt

2022-04-26 19:34:00,599 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:34:00,600 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /home/ma/miniconda3/envs/druglord/bin/gmx
Data prefix:  /home/ma/miniconda3/envs/druglord
Working dir:  /home/ma/git/druglord/temp/6V23
Command line:
  gmx -nobackup -nocopyright mdrun -s 6V23_gppnvt.tpr -o 6V23_nvt.trr -c 6V23_nvt.gro -e 6V23_nvt.edr -g 6V23_nvt.log -cpo 6V23_nvt.cpt

Compiled SIMD: SSE2, but for this host/run AVX_512 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This might 

0

<a id="eqNVTStep3"></a>
### Step 3: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [23]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = pdbCode+'_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_nvt_edr, 
          output_xvg_path=output_nvt_temp_xvg, 
          properties=prop)

2022-04-26 19:34:00,987 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:34:01,016 [MainThread  ] [INFO ]  gmx energy -f /home/ma/git/druglord/temp/6V23/6V23_nvt.edr -o 6V23_nvt_temp.xvg -xvg none < d8a68675-bb86-4d2f-814b-d670421ccb55/instructions.in

2022-04-26 19:34:01,018 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:34:01,019 [MainThread  ] [INFO ]  
Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 299.831       0.19    3.34942    1.27768  (K)

2022-04-26 19:34:01,020 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bol

0

In [24]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

The **npt** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 50,000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = isotropic
-  tau_p = 1.0
-  ref_p = 1.0
-  compressibility = 4.5e-5
-  refcoord_scaling = com
-  gen_vel = no

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **50,000 equilibration steps** with the protein **heavy atoms restrained** (100ps in total), and a Parrinello-Rahman **pressure coupling** algorithm.

*Please refer to the [original example](http://www.mdtutorials.com/gmx/lysozyme/07_equil2.html) for further details.*

In [25]:
# Grompp: Creating portable binary run file for NPT System Equilibration
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = pdbCode+'_gppnpt.tpr'
prop = {
    'mdp':{
        'nsteps':'50000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    },
    'simulation_type': 'npt'
}

# Create and launch bb
grompp(input_gro_path=output_nvt_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppnpt_tpr, 
       input_cpt_path=output_nvt_cpt,  
       properties=prop)

2022-04-26 19:34:01,682 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-26 19:34:01,690 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_genion_top.zip
2022-04-26 19:34:01,690 [MainThread  ] [INFO ]  to:
2022-04-26 19:34:01,691 [MainThread  ] [INFO ]  ['762bb82b-81bf-46b2-a0b7-b0120d3cc938/p2g.top', '762bb82b-81bf-46b2-a0b7-b0120d3cc938/posre.itp']
2022-04-26 19:34:01,692 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:34:01,693 [MainThread  ] [INFO ]  6V23_genion_top.zip
2022-04-26 19:34:01,694 [MainThread  ] [INFO ]  To: 
2022-04-26 19:34:01,694 [MainThread  ] [INFO ]  762bb82b-81bf-46b2-a0b7-b0120d3cc938/p2g.top
2022-04-26 19:34:01,695 [MainThread  ] [INFO ]  762bb82b-81bf-46b2-a0b7-b0120d3cc938/posre.itp
2022-04-26 19:34:01,696 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:34:02,541 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f 2da2b473-527e-410d-9f38-da56963e8b83/grompp.mdp -c 6V23_nvt.gro -r 6V23_nvt.g

0

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [26]:
# Mdrun: Running NPT System Equilibration
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = pdbCode+'_npt.trr'
output_npt_gro = pdbCode+'_npt.gro'
output_npt_edr = pdbCode+'_npt.edr'
output_npt_log = pdbCode+'_npt.log'
output_npt_cpt = pdbCode+'_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr, 
      output_trr_path=output_npt_trr, 
      output_gro_path=output_npt_gro, 
      output_edr_path=output_npt_edr, 
      output_log_path=output_npt_log, 
      output_cpt_path=output_npt_cpt)

2022-04-26 19:34:02,786 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-26 19:34:02,788 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:38:48,250 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 6V23_gppnpt.tpr -o 6V23_npt.trr -c 6V23_npt.gro -e 6V23_npt.edr -g 6V23_npt.log -cpo 6V23_npt.cpt

2022-04-26 19:38:48,252 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:38:48,253 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /home/ma/miniconda3/envs/druglord/bin/gmx
Data prefix:  /home/ma/miniconda3/envs/druglord
Working dir:  /home/ma/git/druglord/temp/6V23
Command line:
  gmx -nobackup -nocopyright mdrun -s 6V23_gppnpt.tpr -o 6V23_npt.trr -c 6V23_npt.gro -e 6V23_npt.edr -g 6V23_npt.log -cpo 6V23_npt.cpt

Compiled SIMD: SSE2, but for this host/run AVX_512 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This might 

0

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [27]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

2022-04-26 19:38:48,691 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:38:48,721 [MainThread  ] [INFO ]  gmx energy -f /home/ma/git/druglord/temp/6V23/6V23_npt.edr -o 6V23_npt_PD.xvg -xvg none < 3b6f3001-a430-45c9-bce1-6cdad4fb3c04/instructions.in

2022-04-26 19:38:48,722 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:38:48,723 [MainThread  ] [INFO ]  
Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 2 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -1.90868          3    102.258   -13.8686  (bar)
Density                     1017.38        0.3    2.39522    2.01201  (kg/m^3)

2022-04-26 19:38:48,724 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer 

0

In [28]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

We can see both of them have converged. Ideally we should check all the thermodynamic parameters:
- temperature
- pressure
- energy
- volume
- density
- RMSD

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein system**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **molecular dynamics parameters (mdp) property** contains the main default parameters to run an **free MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 500,000

In this particular example, the default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **50,000 md steps** (1ns).

In [29]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppmd_tpr = pdbCode+'_gppmd.tpr'
prop = {
    'mdp':{
        'nsteps':'50000',
        #'tc_grps': "DNA Water_and_ions" # NOTE: uncomment this line if working with DNA
    },
    'simulation_type': 'free'
}

# Create and launch bb
grompp(input_gro_path=output_npt_gro, 
       input_top_zip_path=output_genion_top_zip, 
       output_tpr_path=output_gppmd_tpr, 
       input_cpt_path=output_npt_cpt, 
       properties=prop)

2022-04-26 19:38:49,502 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-26 19:38:49,511 [MainThread  ] [INFO ]  Extracting: /home/ma/git/druglord/temp/6V23/6V23_genion_top.zip
2022-04-26 19:38:49,512 [MainThread  ] [INFO ]  to:
2022-04-26 19:38:49,513 [MainThread  ] [INFO ]  ['9a09b67e-b0ee-45b6-a4e3-d22a15c1bfbd/p2g.top', '9a09b67e-b0ee-45b6-a4e3-d22a15c1bfbd/posre.itp']
2022-04-26 19:38:49,514 [MainThread  ] [INFO ]  Unzipping: 
2022-04-26 19:38:49,514 [MainThread  ] [INFO ]  6V23_genion_top.zip
2022-04-26 19:38:49,515 [MainThread  ] [INFO ]  To: 
2022-04-26 19:38:49,516 [MainThread  ] [INFO ]  9a09b67e-b0ee-45b6-a4e3-d22a15c1bfbd/p2g.top
2022-04-26 19:38:49,517 [MainThread  ] [INFO ]  9a09b67e-b0ee-45b6-a4e3-d22a15c1bfbd/posre.itp
2022-04-26 19:38:49,518 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:38:50,023 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright grompp -f b5ed005f-fc1d-4e60-9478-e6b0c9498c4e/grompp.mdp -c 6V23_npt.gro -r 6V23_npt.g

0

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [30]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = pdbCode+'_md.trr'
output_md_gro = pdbCode+'_md.gro'
output_md_edr = pdbCode+'_md.edr'
output_md_log = pdbCode+'_md.log'
output_md_cpt = pdbCode+'_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr, 
      output_trr_path=output_md_trr, 
      output_gro_path=output_md_gro, 
      output_edr_path=output_md_edr, 
      output_log_path=output_md_log, 
      output_cpt_path=output_md_cpt)

2022-04-26 19:38:50,439 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-26 19:38:50,440 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:43:14,036 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 6V23_gppmd.tpr -o 6V23_md.trr -c 6V23_md.gro -e 6V23_md.edr -g 6V23_md.log -cpo 6V23_md.cpt

2022-04-26 19:43:14,039 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:43:14,040 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /home/ma/miniconda3/envs/druglord/bin/gmx
Data prefix:  /home/ma/miniconda3/envs/druglord
Working dir:  /home/ma/git/druglord/temp/6V23
Command line:
  gmx -nobackup -nocopyright mdrun -s 6V23_gppmd.tpr -o 6V23_md.trr -c 6V23_md.gro -e 6V23_md.edr -g 6V23_md.log -cpo 6V23_md.cpt

Compiled SIMD: SSE2, but for this host/run AVX_512 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use. This might affect your 

0

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [31]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmd_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop)

2022-04-26 19:43:14,325 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:43:14,634 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/ma/git/druglord/temp/6V23/6V23_gppmd.tpr -f /home/ma/git/druglord/temp/6V23/6V23_md.trr -o 6V23_rms_first.xvg -xvg none

2022-04-26 19:43:14,636 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:43:14,638 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2022-04-26 19:43:14,639 [MainThread  ] [INFO ]                         :-) GROMACS - gmx rms, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kas

0

In [32]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp, 
          properties=prop)

2022-04-26 19:43:14,866 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:43:15,176 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/ma/git/druglord/temp/6V23/6V23_gppmin.tpr -f /home/ma/git/druglord/temp/6V23/6V23_md.trr -o 6V23_rms_exp.xvg -xvg none

2022-04-26 19:43:15,178 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:43:15,179 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2022-04-26 19:43:15,180 [MainThread  ] [INFO ]                         :-) GROMACS - gmx rms, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kass

0

In [33]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [34]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr, 
          properties=prop)

2022-04-26 19:43:15,668 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:43:15,966 [MainThread  ] [INFO ]  echo "Backbone" | gmx gyrate -s /home/ma/git/druglord/temp/6V23/6V23_gppmin.tpr -f /home/ma/git/druglord/temp/6V23/6V23_md.trr -o 6V23_rgyr.xvg -xvg none

2022-04-26 19:43:15,967 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:43:15,968 [MainThread  ] [INFO ]  Selected 4: 'Backbone'

2022-04-26 19:43:15,968 [MainThread  ] [INFO ]                        :-) GROMACS - gmx gyrate, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Cars

0

In [35]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
}

plotly.offline.iplot(fig)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [36]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_md_trr,
         input_top_path=output_gppmd_tpr,
         output_traj_path=output_imaged_traj, 
          properties=prop)

2022-04-26 19:43:16,585 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:43:16,916 [MainThread  ] [INFO ]  echo "Protein" "Protein" | gmx trjconv -f /home/ma/git/druglord/temp/6V23/6V23_md.trr -s /home/ma/git/druglord/temp/6V23/6V23_gppmd.tpr -fit none -o 6V23_imaged_traj.trr -center -pbc mol -ur compact

2022-04-26 19:43:16,918 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:43:16,919 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'

2022-04-26 19:43:16,920 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan 

0

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [37]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = pdbCode+'_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
         input_top_path=output_gppmd_tpr,
         output_str_path=output_dry_gro, 
          properties=prop)

2022-04-26 19:43:17,186 [MainThread  ] [INFO ]  Not using any container
2022-04-26 19:43:17,557 [MainThread  ] [INFO ]  echo "Protein" | gmx trjconv -f /home/ma/git/druglord/temp/6V23/6V23_md.gro -s /home/ma/git/druglord/temp/6V23/6V23_gppmd.tpr -o 6V23_md_dry.gro

2022-04-26 19:43:17,559 [MainThread  ] [INFO ]  Exit code 0

2022-04-26 19:43:17,560 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 1: 'Protein'

2022-04-26 19:43:17,561 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupino

0

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [38]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

NGLWidget(max_frame=10)

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
